In [1]:
%matplotlib inline
from __future__ import division
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import processData
from settings import *
from mappings import *
from datanetwork import *
from bluetoothdata import *
from features import *

In [3]:
survey_df, sms_call_df, communication_time_graphs = processData.processData()

Loading communication data...
Fixing data format...
Shape of sms-call data: (537575, 5)
Dropping instances where sender and receiver match...
Shape of sms-call data: (533121, 5)
             DateTime  SenderID  ReceiverID  EventType  EventLength
0 2011-08-15 20:03:31     20905       13116          0           15
1 2011-08-15 20:03:37     20905       13116          0            0
2 2011-08-15 20:47:40     75178       30952          0           39
3 2011-08-15 20:47:50     75178       30952          0           23
4 2011-08-15 20:48:52     75178       60830          0           14

Loading survey data...
Fixing data format...
Shape of survey dataframe (199, 59)
   egoid  age_1  hometown_1               ethnicity_1 gender_1  \
0  97900   18.0         6.0           White/Caucasian   Female   
1  97942   19.0         6.0           White/Caucasian     Male   
2  31583   18.0         5.0           White/Caucasian   Female   
3  10281   18.0         6.0  Mexican American/Chicano     Male   
4 

/Users/aastha/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:3855: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


Before:


/Users/aastha/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/aastha/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s



Extracting corresponding communication network...
Number of interactions amongst selected users: 176317
Interactions date range: 2011-08-15 20:47:40 - 2014-08-22 00:23:55

Building network...
Study period: 1
Making network from: 2011-08-15 20:47:40 to 2012-01-28 12:19:54
Range: >= 2011-08-01 00:00:00 and < 2012-01-01 00:00:00
Number of interactions: 28251

Study period: 2
Making network from: 2012-01-28 12:19:54 to 2012-05-09 09:10:55
Range: >= 2012-01-01 00:00:00 and < 2012-05-01 00:00:00
Number of interactions: 30017

Study period: 3
Making network from: 2012-05-09 09:10:55 to 2012-08-20 10:16:29
Range: >= 2012-05-01 00:00:00 and < 2012-08-01 00:00:00
Number of interactions: 9917

Study period: 4
Making network from: 2012-08-20 10:16:29 to 2013-01-18 21:25:53
Range: >= 2012-08-01 00:00:00 and < 2013-01-01 00:00:00
Number of interactions: 34730

Study period: 5
Making network from: 2013-01-18 21:25:53 to 2013-05-02 09:01:05
Range: >= 2013-01-01 00:00:00 and < 2013-05-01 00:00:00
Numb

In [4]:
bluetooth_time_graphs, weekend_bluetooth_time_graphs = getBluetoothNetworks(survey_df)

In [5]:
## Build data

In [6]:
def getTrainTestPairs(communication_time_graphs):
    #t,uid
    train_pairs = list()
    print "Making Train"
    for t in survey_number[:-2]:
        print 't =', t, '-> t =', t+1
        cg = communication_time_graphs[t]
        for uid in cg.nodes():
            next_cg = communication_time_graphs[t+1]
            if next_cg.has_node(uid):
                instance = [t,uid]
                train_pairs.append(instance)
    
    test_pairs = list()
    print "Making Test"
    for t in survey_number[-2:-1]:
        print 't =', t, '-> t =', t+1
        cg = communication_time_graphs[t]
        for uid in cg.nodes():
            next_cg = communication_time_graphs[t+1]
            if next_cg.has_node(uid):
                instance = [t, uid]
                test_pairs.append(instance)
    
    train_pairs = pd.DataFrame(train_pairs, columns=['time', 'uid'])
    test_pairs = pd.DataFrame(test_pairs, columns=['time', 'uid'])
    
    return train_pairs, test_pairs

In [7]:
def computeFeatures(g, i, belief_field):
    f1 = timePredisposition(g, i, belief_field)
    f2 = populationPredisposition(g, i, belief_field)
    f3 = getPopulationBias(g,i, belief_field)
    f4 = getReciprocalBias(g,i, belief_field)

    features = [f1, f2, f3, f4]
    return features

def combinedFeatures(g, bg, i, belief_field):
    #common feature
    f1 = timePredisposition(g, i, belief_field)
    
    #communication network features
    f2 = populationPredisposition(g, i, belief_field)
    f3 = getPopulationBias(g,i, belief_field)
    f4 = getReciprocalBias(g,i, belief_field)

    #bluetooth network features
    f5 = populationPredisposition(bg, i, belief_field)
    f6 = getPopulationBias(bg, i, belief_field)
    f7 = getReciprocalBias(bg, i, belief_field)

    features = [f1,f2,f3,f4,f5,f6,f7]
    return features

In [12]:
##
#1: just communication
#2: just bluetooth
#3: combined
def computeTrainTest(pairs, bluetooth_time_graphs, communication_time_graphs, belief_field, whichNetwork):
    print "Network combo:", whichNetwork
    X = list()
    y = list()    
    for ix, row in pairs.iterrows():
        t = row['time']
        uid = row['uid']

        cg = communication_time_graphs[t]
        bg = bluetooth_time_graphs[t]
        next_cg = communication_time_graphs[t+1]     
        
        if(whichNetwork == 1):
            c_features = computeFeatures(cg, uid, belief_field)
            features = c_features
        elif(whichNetwork == 2):
            b_features = computeFeatures(bg, uid, belief_field)
            features = b_features
        elif(whichNetwork == 3):
            features = combinedFeatures(cg, bg, uid, belief_field)
            
        X.append(features)
        y.append(next_cg.node[uid][belief_field])
    return X, y

def makeTrainTest(train_pairs, test_pairs, bluetooth_time_graphs, communication_time_graphs, belief_field, whichNetwork):
    X_train, y_train = computeTrainTest(train_pairs, bluetooth_time_graphs, communication_time_graphs, belief_field, whichNetwork) 
    X_test, y_test = computeTrainTest(test_pairs, bluetooth_time_graphs, communication_time_graphs, belief_field, whichNetwork) 
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_test = np.array(X_test)
    y_test = np.array(y_test)

    print "Train data shape", X_train.shape, y_train.shape
    print "Test data shape", X_test.shape, y_test.shape
    
    return X_train, y_train, X_test, y_test

def makeData(X_train, y_train, X_test, y_test):
    #make column names
    features_names = list()
    for ix in range(X_train.shape[1]):
        features_names.append('f'+str(ix+1))

    target_names = list()
    if y_train.ndim == 1:
        target_names = ['y']
    else:
        for ix in range(y_train.shape[1]):
            target_names.append('y'+str(ix+1))

    column_names = features_names + target_names

    #turn lists into dataframes
#    print "Train Data"
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)
    train_df = pd.concat([X_train,y_train],axis=1)
    train_df.columns = column_names
#    print train_df.head()

#    print "Test Data"
    X_test = pd.DataFrame(X_test)
    y_test = pd.DataFrame(y_test)
    test_df = pd.concat([X_test,y_test],axis=1)
    test_df.columns = column_names
#    print test_df.head()
    
#     #Drop duplicates if any
#     print "Check for duplicates in train data"
#     print "Train shape before:", train_df.shape
    train_df = pd.DataFrame.drop_duplicates(pd.DataFrame(train_df))
#     print "Train shape after:", train_df.shape
#     print ''

#     print "Test shape before:", test_df.shape
    test_df = pd.DataFrame.drop_duplicates(pd.DataFrame(test_df))
#     print "Test shape after:", test_df.shape
    
    # perform min max normalization on the features
#     print "Min Max Normalization..."
#     scaler = preprocessing.MinMaxScaler()
#     train_df[features_names] = scaler.fit_transform(train_df[features_names])
#     test_df[features_names] = scaler.transform(test_df[features_names])

    return train_df, test_df, features_names, target_names

In [23]:
def fitModel(train_x, train_y, test_x, test_y):
    print "Train data:", train_x.shape, "Test data:", test_x.shape
    clf =  LogisticRegression(random_state = 0)
    clf.fit(train_x, train_y)
#    print clf
    y_hat = clf.predict(test_x)
    f1 = metrics.f1_score(test_y, y_hat, average='weighted')
    acc = metrics.accuracy_score(test_y, y_hat)
    cm = metrics.confusion_matrix(test_y, y_hat)

    print 'F1 %0.2f' % f1
#     print 'Accuracy %0.3f' % acc
    print "Confusion matrix"
    print cm

    return y_hat

def baselines(train_x, train_y, test_x, test_y):
    #Majority
    print "Baseline: Majority Classifier"
    clf1 = DummyClassifier(strategy='most_frequent',random_state=0)
    clf1.fit(train_x, train_y)
    yhat1 = clf1.predict(test_x)
    
    f1 = metrics.f1_score(test_y, yhat1, average='weighted')
    acc = metrics.accuracy_score(test_y, yhat1)
    cm = metrics.confusion_matrix(test_y, yhat1)

    print 'F1 %0.2f' % f1
#     print 'Accuracy %0.2f' % acc
#     print "Confusion matrix"
#     print cm
    
    #Random
    print "Baseline: Random Classifier"
    clf2 = DummyClassifier(strategy='uniform',random_state=0)
    clf2.fit(train_x, train_y)
    yhat2 = clf2.predict(test_x)
    
    f1 = metrics.f1_score(test_y, yhat2, average='weighted')
    acc = metrics.accuracy_score(test_y, yhat2)
    cm = metrics.confusion_matrix(test_y, yhat2)

    print 'F1 %0.2f' % f1
#     print 'Accuracy %0.2f' % acc
#     print "Confusion matrix"
#     print cm
    
    return yhat1, yhat2


In [24]:
train_pairs, test_pairs = getTrainTestPairs(communication_time_graphs)
print "Train:", train_pairs.shape
print "Test:", test_pairs.shape

Making Train
t = 1 -> t = 2
t = 2 -> t = 3
t = 3 -> t = 4
t = 4 -> t = 5
Making Test
t = 5 -> t = 6
Train: (310, 2)
Test: (66, 2)


In [25]:
for whichNetwork in [1,2,3]:
    print whichNetwork
    for belief_field in beliefs:
        print "Analysis for: ", belief_field
        X_train, y_train, X_test, y_test = makeTrainTest(train_pairs, test_pairs, bluetooth_time_graphs, 
                                                                                  communication_time_graphs, 
                                                                                  belief_field, whichNetwork)

        train_df, test_df, features_names, target_names = makeData(X_train, y_train, X_test, y_test)

        print "Train Distribution:", Counter(train_df.y)
        print "Test Distribution:", Counter(test_df.y)
        print features_names, target_names

        print "Model results:"
        fitModel(train_df[features_names], train_df.y, test_df[features_names], test_df.y)

        ## yesterday
        print "Yesterday Model results"
        fitModel(train_df.f1.values.reshape(-1, 1), train_df.y, test_df.f1.values.reshape(-1, 1), test_df.y)
    
        ## Baselines
        print "Baseline Model results:"
        baselines(train_df[features_names], train_df.y, test_df[features_names], test_df.y)
        
        print "-"*50
    print "*"*100


1
Analysis for:  premaritalsex_
Network combo: 1
Network combo: 1
Train data shape (310, 4) (310,)
Test data shape (66, 4) (66,)
Train Distribution: Counter({3: 162, 1: 101, 2: 20})
Test Distribution: Counter({3: 38, 1: 17, 2: 7})
['f1', 'f2', 'f3', 'f4'] ['y']
Model results:
Train data: (283, 4) Test data: (62, 4)
F1 0.79
Confusion matrix
[[16  0  1]
 [ 3  0  4]
 [ 2  0 36]]
Yesterday Model results
Train data: (283, 1) Test data: (62, 1)
F1 0.79
Confusion matrix
[[16  0  1]
 [ 3  0  4]
 [ 2  0 36]]
Baseline Model results:
Baseline: Majority Classifier
F1 0.47
Baseline: Random Classifier
F1 0.32
--------------------------------------------------
Analysis for:  euthanasia_
Network combo: 1
Network combo: 1
Train data shape (310, 4) (310,)
Test data shape (66, 4) (66,)
Train Distribution: Counter({3: 122, 1: 100, 2: 80})
Test Distribution: Counter({3: 28, 1: 20, 2: 18})
['f1', 'f2', 'f3', 'f4'] ['y']
Model results:
Train data: (302, 4) Test data: (66, 4)
F1 0.56
Confusion matrix
[[17  2 

Train data: (206, 1) Test data: (39, 1)
F1 0.51
Confusion matrix
[[ 9  0  1]
 [ 6  0  6]
 [ 2  0 15]]
Baseline Model results:
Baseline: Majority Classifier
F1 0.10
Baseline: Random Classifier
F1 0.31
--------------------------------------------------
Analysis for:  political_
Network combo: 2
Network combo: 2
Train data shape (310, 4) (310,)
Test data shape (66, 4) (66,)
Train Distribution: Counter({3: 65, 1: 62, 2: 55})
Test Distribution: Counter({3: 17, 1: 14, 2: 13})
['f1', 'f2', 'f3', 'f4'] ['y']
Model results:
Train data: (182, 4) Test data: (44, 4)
F1 0.71
Confusion matrix
[[11  3  0]
 [ 5  7  1]
 [ 2  2 13]]
Yesterday Model results
Train data: (182, 1) Test data: (44, 1)
F1 0.49
Confusion matrix
[[11  0  3]
 [ 5  0  8]
 [ 2  0 15]]
Baseline Model results:
Baseline: Majority Classifier
F1 0.22
Baseline: Random Classifier
F1 0.29
--------------------------------------------------
Analysis for:  abortion_
Network combo: 2
Network combo: 2
Train data shape (310, 4) (310,)
Test data 

In [26]:
for whichNetwork in [1,2,3]:
    print whichNetwork
    for belief_field in beliefs:
        print "Analysis for: ", belief_field
        X_train, y_train, X_test, y_test = makeTrainTest(train_pairs, test_pairs, 
                                                         weekend_bluetooth_time_graphs, 
                                                         communication_time_graphs, belief_field, whichNetwork)

        train_df, test_df, features_names, target_names = makeData(X_train, y_train, X_test, y_test)

        print "Train Distribution:", Counter(train_df.y)
        print "Test Distribution:", Counter(test_df.y)

        print "Model results:"
        fitModel(train_df[features_names], train_df.y, test_df[features_names], test_df.y)

        
        ## yesterday
        print "Yesterday Model results"
        fitModel(train_df.f1.values.reshape(-1, 1), train_df.y, test_df.f1.values.reshape(-1, 1), test_df.y)
    
        ## Baselines
        print "Baseline Model results:"
        baselines(train_df[features_names], train_df.y, test_df[features_names], test_df.y)
        
        print "-"*50
    print "*"*100

1
Analysis for:  premaritalsex_
Network combo: 1
Network combo: 1
Train data shape (310, 4) (310,)
Test data shape (66, 4) (66,)
Train Distribution: Counter({3: 162, 1: 101, 2: 20})
Test Distribution: Counter({3: 38, 1: 17, 2: 7})
Model results:
Train data: (283, 4) Test data: (62, 4)
F1 0.79
Confusion matrix
[[16  0  1]
 [ 3  0  4]
 [ 2  0 36]]
Yesterday Model results
Train data: (283, 1) Test data: (62, 1)
F1 0.79
Confusion matrix
[[16  0  1]
 [ 3  0  4]
 [ 2  0 36]]
Baseline Model results:
Baseline: Majority Classifier
F1 0.47
Baseline: Random Classifier
F1 0.32
--------------------------------------------------
Analysis for:  euthanasia_
Network combo: 1
Network combo: 1
Train data shape (310, 4) (310,)
Test data shape (66, 4) (66,)
Train Distribution: Counter({3: 122, 1: 100, 2: 80})
Test Distribution: Counter({3: 28, 1: 20, 2: 18})
Model results:
Train data: (302, 4) Test data: (66, 4)
F1 0.56
Confusion matrix
[[17  2  1]
 [ 9  2  7]
 [ 6  1 21]]
Yesterday Model results
Train dat

Train data: (180, 4) Test data: (44, 4)
F1 0.71
Confusion matrix
[[11  3  0]
 [ 5  7  1]
 [ 2  2 13]]
Yesterday Model results
Train data: (180, 1) Test data: (44, 1)
F1 0.49
Confusion matrix
[[11  0  3]
 [ 5  0  8]
 [ 2  0 15]]
Baseline Model results:
Baseline: Majority Classifier
F1 0.22
Baseline: Random Classifier
F1 0.29
--------------------------------------------------
Analysis for:  abortion_
Network combo: 2
Network combo: 2
Train data shape (310, 4) (310,)
Test data shape (66, 4) (66,)
Train Distribution: Counter({1: 78, 3: 67, 2: 28})
Test Distribution: Counter({1: 19, 3: 16, 2: 8})
Model results:
Train data: (173, 4) Test data: (43, 4)
F1 0.54
Confusion matrix
[[15  0  4]
 [ 3  0  5]
 [ 5  0 11]]
Yesterday Model results
Train data: (173, 1) Test data: (43, 1)
F1 0.54
Confusion matrix
[[15  0  4]
 [ 3  0  5]
 [ 5  0 11]]
Baseline Model results:
Baseline: Majority Classifier
F1 0.27
Baseline: Random Classifier
F1 0.27
--------------------------------------------------
Analysis 